In [17]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Load the CSV file
csv_file = '/Users/joshmanto/Downloads/BB/tree-semantics/lats-longs-csv/locations.csv'
data = pd.read_csv(csv_file)

# Display the first few rows of the data
print(data.head())

   longitude   latitude
0  28.029352 -26.248023
1  28.009093 -26.207037
2  28.028533 -26.136593
3  28.040460 -26.133520
4  27.587758 -26.040437


CLUSTER LAT AND LONG USING DBSCAN SCIKIT 
STORE AS VARIABLES TO BE SENT AS INPUT 

In [3]:
from sklearn.cluster import DBSCAN
import numpy as np

# Extract latitude and longitude
coords = data[['latitude', 'longitude']].values

# Define the DBSCAN clustering parameters
epsilon = 0.01  # Adjust the value as needed
min_samples = 5

# Apply DBSCAN clustering
db = DBSCAN(eps=epsilon, min_samples=min_samples, metric='haversine').fit(np.radians(coords))
labels = db.labels_

# Add the cluster labels to the data
data['cluster'] = labels

# Display the clustered data
print(data.head(50))

    longitude   latitude  cluster
0   28.029352 -26.248023        0
1   28.009093 -26.207037        0
2   28.028533 -26.136593        0
3   28.040460 -26.133520        0
4   27.587758 -26.040437        0
5   28.120983 -25.489502       -1
6   25.865823 -24.669195       -1
7   25.875787 -24.654352       -1
8   57.462418 -20.234445       -1
9   57.582078 -20.096572       -1
10  57.584127 -20.067255       -1
11 -13.411811  27.097262       -1
12  -9.547540  30.352274       -1
13  -8.028289  31.621362       -1
14  -9.227294  32.276417       -1
15  -7.657850  33.522778       -1
16  -7.660924  33.528330       -1
17  -7.608301  33.566550       -1
18  -7.532156  33.615298       -1
19  -6.871319  33.951440       -1
20  -6.733552  33.995864       -1
21  -1.917905  34.661148       -1
22  -1.895867  34.707470       -1
23  10.748792  36.472305       -1
24  10.085372  36.830607       -1
25  28.919330 -31.381882       -1
26  27.219763 -30.600395       -1
27  28.515602 -30.351668       -1
28  27.437025 

In [4]:
# Store clustered coordinates with increasing filenames
clustered_coords = {}

for cluster_label in np.unique(labels):
    if cluster_label != -1:  # Ignore noise points
        cluster_data = data[data['cluster'] == cluster_label]
        clustered_coords[f'AOI{cluster_label + 1}'] = cluster_data[['latitude', 'longitude']].values.tolist()

# Display the clustered coordinates
for aoi, coords in clustered_coords.items():
    print(f"{aoi}: {coords}")


AOI1: [[-26.248023333333336, 28.02935166666667], [-26.207036666666667, 28.009093333333333], [-26.136593333333337, 28.028533333333332], [-26.133519999999997, 28.04046], [-26.040436666666665, 27.587758333333333], [-25.56795833333333, 27.246971666666663]]
AOI2: [[-29.31815, 27.558531666666667], [-29.119956667, 27.87818], [-29.03846, 27.781255], [-28.935909999999996, 28.154376666666668], [-28.86071833333333, 28.042841666666664], [-28.667095000000003, 28.075698333333335]]
AOI3: [[-27.83289333333333, 31.767108333333333], [-27.30445, 31.88127667], [-27.300715, 31.51146175], [-27.1869663, 31.345925516666664], [-27.080904900000004, 31.926145150000004], [-27.05752167, 31.59494667], [-27.00534612, 31.12097247], [-26.91165738333333, 31.64543821666667], [-26.7792956, 31.75505041666667], [-26.6655, 31.9886983333333], [-26.25189221666667, 31.638282366666665]]
AOI4: [[-9.012521667, 13.334003333], [-8.94969166666667, 13.0656583333333], [-8.874646667, 13.195736667], [-8.86241333333333, 13.2561833333333]

AUTHENTICATION AND ENDPOINTS 

In [5]:
import requests
import json
from datetime import datetime

# API key for Planet.com
API_KEY = 'PLAK38d647437d504a2da7e86373e0356771'
auth = (API_KEY, '')

# Base URL for the Planet API
BASE_URL = "https://api.planet.com/data/v1"


PARAMETERS AND FILTERS. 

In [6]:
from shapely.geometry import box, mapping
import datetime

# TIME AND /mnt/data/locations.csv PARAMETER DEFAULTS 
start_date = "2020-01-01T00:00:00.000Z"
end_date = "2020-02-01T00:00:00.000Z"
max_cloud_cover = 0.5

# HELPER (CREATE BOUNDING BOX) 
def create_bounding_box(coords):
    lats, lons = zip(*coords)
    return box(min(lons), min(lats), max(lons), max(lats))

# MAIN FUNCTION. SOURCES HELPER. 
def create_filters(coords):
    bounding_box = create_bounding_box(coords)
    geojson_geometry = mapping(bounding_box)
    
    geometry_filter = {
        "type": "GeometryFilter",
        "field_name": "geometry",
        "config": geojson_geometry
    }

    date_range_filter = {
        "type": "DateRangeFilter",
        "field_name": "acquired",
        "config": {
            "gte": start_date,
            "lte": end_date
        }
    }

    cloud_cover_filter = {
        "type": "RangeFilter",
        "field_name": "cloud_cover",
        "config": {
            "lte": max_cloud_cover
        }
    }

    combined_filter = {
        "type": "AndFilter",
        "config": [geometry_filter, date_range_filter, cloud_cover_filter]
    }
    
    return combined_filter


In [7]:
#CREATE FILTER FOR FIRST TWO AOIs ONLY. 
filters_AOI1 = create_filters(clustered_coords['AOI1'])
filters_AOI2 = create_filters(clustered_coords['AOI2'])

In [25]:
import json
#test to see if the filters and AOIs are tagged properly
print("Filter for AOI1:")
print(json.dumps(filters_AOI1, indent=2))

print("\nFilter for AOI2:")
print(json.dumps(filters_AOI2, indent=2))

Filter for AOI1:
{
  "type": "AndFilter",
  "config": [
    {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              28.04046,
              -26.248023333333336
            ],
            [
              28.04046,
              -25.56795833333333
            ],
            [
              27.246971666666663,
              -25.56795833333333
            ],
            [
              27.246971666666663,
              -26.248023333333336
            ],
            [
              28.04046,
              -26.248023333333336
            ]
          ]
        ]
      }
    },
    {
      "type": "DateRangeFilter",
      "field_name": "acquired",
      "config": {
        "gte": "2020-01-01T00:00:00.000Z",
        "lte": "2020-02-01T00:00:00.000Z"
      }
    },
    {
      "type": "RangeFilter",
      "field_name": "cloud_cover",
      "config": {
        "lte": 0.5
      }

POST REQUEST THAT WILL PRINT THE FIRST IN THE LIST OF RESULTS 

In [26]:
from requests.auth import HTTPBasicAuth
# Function to send POST request
def send_post_request(filters, item_type="PSScene"):
    search_request = {
        "item_types": [item_type],
        "filter": filters
    }
    
    search_result = requests.post(
        'https://api.planet.com/data/v1/quick-search',
        auth=HTTPBasicAuth(API_KEY, ''),
        json=search_request
    )
    
    geojson = search_result.json()
    
    # Print the first result
    if 'features' in geojson and geojson['features']:
        print(json.dumps(geojson['features'][0], indent=2))
    else:
        print("No results found.")


# Send POST requests
print("Results for AOI1:")
send_post_request(filters_AOI1)

print("\nResults for AOI2:")
send_post_request(filters_AOI2)


Results for AOI1:
{
  "_links": {
    "_self": "https://api.planet.com/data/v1/item-types/PSScene/items/20200105_092927_42_1069",
    "assets": "https://api.planet.com/data/v1/item-types/PSScene/items/20200105_092927_42_1069/assets/",
    "thumbnail": "https://tiles.planet.com/data/v1/item-types/PSScene/items/20200105_092927_42_1069/thumb"
  },
  "_permissions": [
    "assets.basic_analytic_4b:download",
    "assets.basic_analytic_4b_rpc:download",
    "assets.basic_analytic_4b_xml:download",
    "assets.basic_udm2:download",
    "assets.ortho_analytic_4b:download",
    "assets.ortho_analytic_4b_sr:download",
    "assets.ortho_analytic_4b_xml:download",
    "assets.ortho_udm2:download",
    "assets.ortho_visual:download"
  ],
  "assets": [
    "basic_analytic_4b",
    "basic_analytic_4b_rpc",
    "basic_analytic_4b_xml",
    "basic_udm2",
    "ortho_analytic_4b",
    "ortho_analytic_4b_sr",
    "ortho_analytic_4b_xml",
    "ortho_udm2",
    "ortho_visual"
  ],
  "geometry": {
    "coor

POST REQUEST THAT WILL SEND ALL THE AVAILABLE RESULTS

In [ ]:
def send_post_request(filters, item_type="PSScene"):
    search_request = {
        "item_types": [item_type],
        "filter": filters
    }
    
    search_result = requests.post(
        'https://api.planet.com/data/v1/quick-search',
        auth=HTTPBasicAuth(API_KEY, ''),
        json=search_request
    )
    
    geojson = search_result.json()
    
    if 'features' in geojson and geojson['features']:
        total_results = len(geojson['features'])
        print(f"Total number of results: {total_results}")
        for feature in geojson['features']:
            print(json.dumps(feature, indent=2))
        return geojson['features']
    else:
        print("No results found.")
        return []

# Send POST requests and get results
print("Results for AOI1:")
features_AOI1 = send_post_request(filters_AOI1)

print("\nResults for AOI2:")
features_AOI2 = send_post_request(filters_AOI2)


POST REQUEST THAT WILL LIST THE TOTAL NUMBER OF RESULTS ONLY. 

In [39]:
# Function to send POST request and get total results count
def send_post_request(filters, item_type="PSScene"):
    search_request = {
        "item_types": [item_type],
        "filter": filters
    }
    
    search_result = requests.post(
        'https://api.planet.com/data/v1/quick-search',
        auth=HTTPBasicAuth(API_KEY, ''),
        json=search_request
    )
    
    geojson = search_result.json()
    
    if 'features' in geojson and geojson['features']:
        total_results = len(geojson['features'])
        print(f"Total number of results: {total_results}")
        return geojson['features']
    else:
        print("No results found.")
        return []

# Send POST requests and get total results count
print("Results for AOI1:")
features_AOI1 = send_post_request(filters_AOI1)

print("\nResults for AOI2:")
features_AOI2 = send_post_request(filters_AOI2)


Results for AOI1:
Total number of results: 250

Results for AOI2:
Total number of results: 250


In [40]:
# Function to retrieve metadata for a single image ID
def get_image_metadata(image_id, item_type="PSScene"):
    id_url = f'https://api.planet.com/data/v1/item-types/{item_type}/items/{image_id}/assets'
    
    result = requests.get(
        id_url,
        auth=HTTPBasicAuth(API_KEY, '')
    )
    
    # List of asset types available for this particular satellite image
    print(f"Metadata for image ID {image_id}:")
    print(result.json().keys())

In [41]:
if features_AOI1:
    image_ids_AOI1 = [feature['id'] for feature in features_AOI1]
    print(f"Image IDs for AOI1: {image_ids_AOI1}")
    # Retrieve metadata for the first image ID
    if image_ids_AOI1:
        get_image_metadata(image_ids_AOI1[0]) #edit this to generate the correct image metadata for any of the 250 image ids. 

# Extract and print image IDs for AOI2
if features_AOI2:
    image_ids_AOI2 = [feature['id'] for feature in features_AOI2]
    print(f"Image IDs for AOI2: {image_ids_AOI2}")
    # Retrieve metadata for the first image ID
    if image_ids_AOI2:
        get_image_metadata(image_ids_AOI2[0]) #edit this 

Image IDs for AOI1: ['20200105_092927_42_1069', '20200105_092925_39_1069', '20200126_082142_96_105d', '20200126_082138_89_105d', '20200126_082140_92_105d', '20200123_075829_1014', '20200123_075828_1014', '20200123_075826_1014', '20200123_075827_1014', '20200123_075824_1014', '20200123_075825_1014', '20200123_075822_1014', '20200123_075820_1014', '20200123_075823_1014', '20200123_075818_1014', '20200123_075821_1014', '20200123_075819_1014', '20200103_075812_1014', '20200103_075815_1014', '20200103_075810_1014', '20200103_075818_1014', '20200103_075816_1014', '20200103_075811_1014', '20200103_075813_1014', '20200103_075808_1014', '20200103_075807_1014', '20200103_075814_1014', '20200103_075817_1014', '20200103_075809_1014', '20200129_072925_0f46', '20200129_072928_0f46', '20200129_072924_0f46', '20200129_072920_0f46', '20200129_072919_0f46', '20200129_072926_0f46', '20200129_072929_0f46', '20200129_072923_0f46', '20200129_072920_1_0f46', '20200129_072922_0f46', '20200129_072921_0f46', '2

Asset Activation and Download Link generator

In [42]:
import requests
from requests.auth import HTTPBasicAuth
from tqdm import tqdm
import time

def activate_asset1(item_id, asset_type='analytic'):
    item_type = 'PSScene'
    asset_url = f'https://api.planet.com/data/v1/item-types/{item_type}/items/{item_id}/assets/'
    
    # Request to get the asset details
    result = requests.get(asset_url, auth=HTTPBasicAuth(API_KEY, ''))
    asset_status = result.json()
    
    # Print the current status of the asset
    current_status = asset_status[asset_type]['status']
    print(f"Current status of asset {item_id}: {current_status}")
    
    if current_status != 'active':
        # Parse out useful links
        links = asset_status[asset_type]['_links']
        activation_link = links['activate']
        
        # Request activation of the asset
        print(f"Activating asset {item_id}...")
        activate_result = requests.get(activation_link, auth=HTTPBasicAuth(API_KEY, ''))
        
        if activate_result.status_code == 202:
            print(f"Activation request for asset {item_id} has been submitted successfully.")
        else:
            print(f"Failed to submit activation request for asset {item_id}. Status code: {activate_result.status_code}")
            return False

        # Check activation status with a progress bar
        with tqdm(total=100, desc="Activating", bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]") as pbar:
            while True:
                time.sleep(10)  # Wait for 10 seconds before checking the status again
                result = requests.get(asset_url, auth=HTTPBasicAuth(API_KEY, ''))
                asset_status = result.json()
                current_status = asset_status[asset_type]['status']
                
                if current_status == 'active':
                    print(f"Asset {item_id} is now active.")
                    break
                elif current_status == 'activating':
                    pbar.update(10)  # Update the progress bar
                else:
                    print(f"Asset {item_id} status: {current_status}")
                    break
    else:
        print(f"Asset {item_id} is already active.")

Usage 

In [ ]:
#USAGE
# item_id = '20200128_075448_1009'  # Replace with your selected image ID
# asset_type = 'basic_analytic_4b'  # You can change this to any other asset type if needed


# Example usage to activate an asset and get the download link
if image_ids_AOI1:
    selected_image_id = image_ids_AOI1[2]  # Replace with the desired image ID
    activate_asset1(selected_image_id, asset_type='basic_analytic_4b')
    
    

LOG of assets already activated: 

20200105_092927_42_1069
20200105_092925_39_1069
20200126_082142_96_105d


to solve the activation time, i will try to employ two solutions. (1) we can use multiprocessing to send multiple requests at the same time. (2) i will try to use batch request to send multiple image for activation in a single batch. 

an optimization that I think you can make on your part is use DUKE vpn given planet is based in the US.

#an implementation that we would like to make as we refine this piece of code is to create a LOG of all the assets that are already activated. 
#another implementation is to use a for loop code to log send the activation request through the entire list. 
#another implementation is to create a log that will all activated assets. This should be in the helper code itself. 
#another implementation is to add other API keys and send requests simultanously


INFO about rate limiting: 

Rate Limiting
To improve the experience for all of our users, Planet uses rate limiting to prevent overloading the system. If handled correctly, rate limiting errors can be a normal and useful part of working with the API.

When a rate limit has been exceeded, the Planet API responds with an HTTP 429 response code. When this occurs, we recommend implementing retry with an exponential backoff. An exponential backoff means that you wait for exponentially longer intervals between each retry of a single failing request.

The following rate limit is currently in place:

Tasking endpoint: 10 requests per second, per API key.
Maximum Payload Size¶
When sending a POST request to the Planet API, the server will accept a maximum payload size of 1 megabyte.


In [43]:
import requests
from requests.auth import HTTPBasicAuth
from tqdm import tqdm
import time

# Your Planet API key
API_KEY = 'PLAKe8a128a493104644888a58e5a0b4b780'

def activate_asset(item_id, asset_type='analytic'):
    item_type = 'PSScene'
    asset_url = f'https://api.planet.com/data/v1/item-types/{item_type}/items/{item_id}/assets/'
    
    # Request to get the asset details
    result = requests.get(asset_url, auth=HTTPBasicAuth(API_KEY, ''))
    asset_status = result.json()
    
    # Print the current status of the asset
    current_status = asset_status[asset_type]['status']
    print(f"Current status of asset {item_id}: {current_status}")
    
    if current_status != 'active':
        # Parse out useful links
        links = asset_status[asset_type]['_links']
        activation_link = links['activate']
        
        # Request activation of the asset
        print(f"Activating asset {item_id}...")
        activate_result = requests.get(activation_link, auth=HTTPBasicAuth(API_KEY, ''))
        
        if activate_result.status_code == 202:
            print(f"Activation request for asset {item_id} has been submitted successfully.")
        else:
            print(f"Failed to submit activation request for asset {item_id}. Status code: {activate_result.status_code}")
            return False

        # Check activation status
        while True:
            time.sleep(10)  # Wait for 10 seconds before checking the status again
            result = requests.get(asset_url, auth=HTTPBasicAuth(API_KEY, ''))
            asset_status = result.json()
            current_status = asset_status[asset_type]['status']
            
            if current_status == 'active':
                print(f"Asset {item_id} is now active.")
                return True
            elif current_status == 'activating':
                print(f"Asset {item_id} status: activating")
            else:
                print(f"Asset {item_id} status: {current_status}")
                return False
    else:
        print(f"Asset {item_id} is already active.")
        return True

def is_activated(image_id):
    # Check if image is already activated
    if image_id in activated_images:
        print(f"Image {image_id} is already activated.")
        return

    # Activate asset
    activate_asset(image_id, asset_type='basic_analytic_4b')



In [44]:
if features_AOI1:
    image_ids_AOI1 = [feature['id'] for feature in features_AOI1]
    print(f"Image IDs for AOI1: {image_ids_AOI1}")
    # Retrieve metadata for the first image ID
    if image_ids_AOI1:
        get_image_metadata(image_ids_AOI1[0]) #edit this to generate the correct image metadata for any of the 250 image ids. 

Image IDs for AOI1: ['20200105_092927_42_1069', '20200105_092925_39_1069', '20200126_082142_96_105d', '20200126_082138_89_105d', '20200126_082140_92_105d', '20200123_075829_1014', '20200123_075828_1014', '20200123_075826_1014', '20200123_075827_1014', '20200123_075824_1014', '20200123_075825_1014', '20200123_075822_1014', '20200123_075820_1014', '20200123_075823_1014', '20200123_075818_1014', '20200123_075821_1014', '20200123_075819_1014', '20200103_075812_1014', '20200103_075815_1014', '20200103_075810_1014', '20200103_075818_1014', '20200103_075816_1014', '20200103_075811_1014', '20200103_075813_1014', '20200103_075808_1014', '20200103_075807_1014', '20200103_075814_1014', '20200103_075817_1014', '20200103_075809_1014', '20200129_072925_0f46', '20200129_072928_0f46', '20200129_072924_0f46', '20200129_072920_0f46', '20200129_072919_0f46', '20200129_072926_0f46', '20200129_072929_0f46', '20200129_072923_0f46', '20200129_072920_1_0f46', '20200129_072922_0f46', '20200129_072921_0f46', '2

In [46]:
if __name__ == "__main__":
    # Load the list of already activated images from a log file or a list
    activated_images = [
        "20200105_092927_42_1069",
        "20200105_092925_39_1069",
        "20200126_082142_96_105d"
    ]

    # List of image IDs to activate
    # Assuming features_AOI1 is defined somewhere 
    image_ids = [feature['id'] for feature in features_AOI1[:15]]  # Limit to 150 for this example

    # Process each image ID sequentially with progress tracking
    for image_id in tqdm(image_ids, desc="Activating images"):
        is_activated(image_id)

Activating images:   0%|                                                                                                                                                                                                     | 0/15 [00:00<?, ?it/s]

Image 20200105_092927_42_1069 is already activated.
Image 20200105_092925_39_1069 is already activated.
Image 20200126_082142_96_105d is already activated.
Current status of asset 20200126_082138_89_105d: inactive
Activating asset 20200126_082138_89_105d...


Activating images:  27%|██████████████████████████████████████████████████▍                                                                                                                                          | 4/15 [00:02<00:05,  1.88it/s]

Failed to submit activation request for asset 20200126_082138_89_105d. Status code: 403
Current status of asset 20200126_082140_92_105d: inactive
Activating asset 20200126_082140_92_105d...


Activating images:  33%|███████████████████████████████████████████████████████████████                                                                                                                              | 5/15 [00:04<00:09,  1.09it/s]

Failed to submit activation request for asset 20200126_082140_92_105d. Status code: 403
Current status of asset 20200123_075829_1014: inactive
Activating asset 20200123_075829_1014...


Activating images:  40%|███████████████████████████████████████████████████████████████████████████▌                                                                                                                 | 6/15 [00:06<00:11,  1.27s/it]

Failed to submit activation request for asset 20200123_075829_1014. Status code: 403
Current status of asset 20200123_075828_1014: inactive
Activating asset 20200123_075828_1014...


Activating images:  40%|███████████████████████████████████████████████████████████████████████████▌                                                                                                                 | 6/15 [00:07<00:11,  1.29s/it]


KeyboardInterrupt: 

In [ ]:
#edit 1: parsing for both Cluster.py and Query.py are separate now
#edit 2: convert ndarray into a list. Filetype result is ndarray from Cluster.py, then convert into a list which can be sent as JSON payload
#NOTE: tolist() function to convert from ndarray to list, and geojson.variable.json() to convert into JSON payload